# DATATHON - Trabalho Final MLET - Grupo 19 - Cold Start

## 1. Apresentação  

### Objetivo

Seu desafio é desenvolver um modelo de sistema de recomendação e realizar o deploy dele utilizando as técnicas aprendidas no curso. Nesse cenário, surge o desafio de fornecer recomendações personalizadas para cada usuário com base nos dados de notícias do DPlay, predizendo qual será a próxima notícia que ele vai ler.

Técnica Aprendizagem supervisionada, mais precisamente a técnicas de recomendação.

### Conjunto de Treino 

O conjunto de treino está disponibilizado em diferentes partes, cada uma contendo informação complementar.
Os arquivos treino_parte_X.csv, em que X é um valor de 1 até 6, consistem das colunas:
    
    
    1. userId: id do usuário.
    2. userType: usuário logado ou anônimo.
    3. HistorySize: quantidade de notícias lidas pelo usuário.
    4. history: lista de notícias visitadas pelo usuário.
    5. TimestampHistory: momento em que o usuário visitou a página.
    6. timeOnPageHistory: quantidade de ms em que o usuário ficou na página.
    7. numberOfClicksHistory: quantidade de clicks na matéria.
    8. scrollPercentageHistory: quanto o usuário visualizou da matéria.
    9. pageVisitsCountHistory: quantidade de vezes que o usuário visitou a matéria.


Além desses arquivos, a pasta de treino contém uma subpasta denominada de itens. Ela contém a seguinte informação:

    1. Page: id da matéria. Esse é o mesmo id que aparece na coluna history de antes.
    2. Url: url da matéria.
    3. Issued: data em que a matéria foi criada.
    4. Modified: última data em que a matéria foi modificada.
    5. Title: título da matéria.
    6. Body: corpo da matéria.Caption: subtítulo da matéria.

Este conjunto de treino consiste em dados de usuários reais da DPlay. Eles forem coletados até uma data limite T (a maior data em todo o conjunto TimestampHistory).


### Conjunto de Validação

Capturando informações de um período posterior ao treino, ou seja, não existe sobreposição temporal com o treino, o conjunto de validação consiste em:

    1. userId: id do usuário.
    2. userType: usuário logado ou anônimo.
    3. history: lista de páginas que devem ser recomendadas ao usuário.



O seu objetivo é gerar um ranking para a coluna history. Ou seja: quando um usuário loga, é necessário prever quais serão os próximos acessos dele. Observe que o mesmo userId está tanto na validação quanto no treino.

## 2. Carregamento dos Dados

In [1]:
import pandas as pd
import numpy as np

## Configurações da AWS - S3

In [2]:
import os
import boto3
import pickle
from io import BytesIO
from kaggle_secrets import UserSecretsClient
from botocore.exceptions import NoCredentialsError

user_secrets = UserSecretsClient()

# Configurações da AWS
AWS_ACCESS_KEY = user_secrets.get_secret("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = user_secrets.get_secret("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = user_secrets.get_secret("AWS_SESSION_TOKEN")
AWS_REGION = user_secrets.get_secret("AWS_REGION")
BUCKET_NAME = "datathon-base"

# Criar conexão com o S3
def get_s3_client():
    return boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        aws_session_token=AWS_SESSION_TOKEN,
        region_name=AWS_REGION,
    )

def upload_to_s3(file_name, bucket_name, object_name=None):
    """
    Faz upload de um arquivo CSV para um bucket S3.

    :param file_name: Caminho do arquivo local a ser enviado
    :param bucket_name: Nome do bucket S3
    :param object_name: Nome do arquivo no S3 (opcional, usa o nome original se None)
    :param aws_access_key: Chave de acesso AWS (opcional se já configurado)
    :param aws_secret_key: Chave secreta AWS (opcional se já configurado)
    """
    try:
        # Criar cliente S3
        s3_client = get_s3_client()
        
        if object_name is None:
            object_name = file_name.split("/")[-1]  # Usa apenas o nome do arquivo se não especificado
        
        s3_client.upload_file(file_name, bucket_name, object_name)
        print(f"Upload de {file_name} para {bucket_name}/{object_name} concluído com sucesso.")
    except NoCredentialsError:
        print("Credenciais não encontradas. Verifique suas configurações de autenticação AWS.")
    except Exception as e:
        print(f"Erro ao fazer upload: {e}")
        
def upload_to_s3(file_path, bucket_name=BUCKET_NAME, object_name=None):
    """Faz upload de um arquivo local para um bucket S3."""
    
    if object_name is None:
        object_name = os.path.basename(file_path)  # Nome padrão do arquivo no S3

    try:
        s3_client = get_s3_client()

        s3_client.upload_file(file_path, bucket_name, object_name)
        print(f"✔ Upload de {file_path} para s3://{bucket_name}/{object_name} concluído.")
        return True

    except FileNotFoundError:
        print("❌ O arquivo não foi encontrado.")
        return False
    except NoCredentialsError:
        print("❌ Credenciais da AWS não encontradas.")
        return False

# Função para salvar o modelo no S3
def save_model_to_s3(model_data, s3_path="models/news_recommendation_model_cold_start.pkl"):
    s3 = get_s3_client()
    
    # Serializar o modelo
    with open("news_recommendation_model_cold_start.pkl", "wb") as f:
        pickle.dump(model_data, f)

    # Fazer upload para o S3
    with open("news_recommendation_model_cold_start.pkl", "rb") as f:
        s3.upload_fileobj(f, BUCKET_NAME, s3_path)

    print(f"Modelo salvo no S3 em: s3://{BUCKET_NAME}/{s3_path}")

# Função para carregar o modelo do S3
def load_model_from_s3(s3_path="models/news_recommendation_model_cold_start.pkl"):
    s3 = get_s3_client()
    
    # Baixar o arquivo do S3
    with open("news_recommendation_model_cold_start.pkl", "wb") as f:
        s3.download_fileobj(BUCKET_NAME, s3_path, f)

    # Carregar o modelo
    with open("news_recommendation_model_cold_start.pkl", "rb") as f:
        model_data = pickle.load(f)

    print("Modelo Cold Start carregado com sucesso!")
    return model_data

def download_dataframe_from_s3(filename):
    """
    Baixa um arquivo Parquet do S3 e carrega como DataFrame.
    
    :param filename: Nome do arquivo no S3 (ex: "dados/meu_arquivo.parquet")
    :return: DataFrame do pandas
    """
    try:
        s3 = get_s3_client()
        response = s3.get_object(Bucket=BUCKET_NAME, Key=filename)
        buffer = BytesIO(response["Body"].read())
        df = pd.read_parquet(buffer, engine="pyarrow")
        print(f"Arquivo {filename} baixado com sucesso!")
        return df
    except Exception as e:
        print("Erro ao baixar arquivo do S3:", str(e))
        return None

def list_s3_files(bucket_name, prefix=''):
    """
    Lista os arquivos de um bucket S3.
    
    :param bucket_name: Nome do bucket
    :param prefix: Prefixo opcional para filtrar os arquivos
    :return: Lista de arquivos no bucket
    """
    s3 = get_s3_client() 
    files = []
    
    try:
        response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)
        if 'Contents' in response:
            files = [obj['Key'] for obj in response['Contents']]
    except Exception as e:
        print(f"Erro ao listar arquivos do bucket {bucket_name}: {e}")
    
    return files

print("Done")

Done


### Listando arquivos no Bucket - S3

In [3]:
arquivos = list_s3_files(BUCKET_NAME)
arquivos


['base_original/interacoes/dados_2022_07.csv',
 'base_original/interacoes/dados_2022_08.csv',
 'base_original/noticias/dados_2014_07.csv',
 'base_original/noticias/dados_2015_12.csv',
 'base_original/noticias/dados_2016_10.csv',
 'base_original/noticias/dados_2016_11.csv',
 'base_original/noticias/dados_2016_12.csv',
 'base_original/noticias/dados_2017_01.csv',
 'base_original/noticias/dados_2017_02.csv',
 'base_original/noticias/dados_2017_03.csv',
 'base_original/noticias/dados_2017_04.csv',
 'base_original/noticias/dados_2017_05.csv',
 'base_original/noticias/dados_2017_06.csv',
 'base_original/noticias/dados_2017_07.csv',
 'base_original/noticias/dados_2017_08.csv',
 'base_original/noticias/dados_2017_09.csv',
 'base_original/noticias/dados_2017_10.csv',
 'base_original/noticias/dados_2017_11.csv',
 'base_original/noticias/dados_2017_12.csv',
 'base_original/noticias/dados_2018_01.csv',
 'base_original/noticias/dados_2018_02.csv',
 'base_original/noticias/dados_2018_03.csv',
 'base

In [4]:
class S3FileDownloader:
    def __init__(self, bucket_name, prefix, local_dir='downloads', arq ='', new_arq =''):
        """
        Classe para baixar arquivos de um diretório no S3 e concatená-los em um único DataFrame.
        
        :param bucket_name: Nome do bucket S3
        :param prefix: Prefixo do diretório no S3
        :param local_dir: Diretório local para salvar os arquivos (padrão: 'downloads')
        """
        self.s3_client = get_s3_client()
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.local_dir = local_dir
        self.arq = arq
        self.new_arq = new_arq
        
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)
    
    def list_s3_files(self):
        """Lista os arquivos CSV no bucket S3 dentro do diretório especificado."""
        files = []
        try:
            response = self.s3_client.list_objects_v2(Bucket=self.bucket_name, Prefix=self.prefix)
            if 'Contents' in response:
                files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.csv')]
        except Exception as e:
            print(f"Erro ao listar arquivos do bucket {self.bucket_name}: {e}")
        return files
    
    def download_files(self):
        """Baixa todos os arquivos CSV listados do S3 para o diretório local."""
        files = self.list_s3_files()
        for file_key in files:
            file_name = os.path.join(self.local_dir, os.path.basename(file_key))
            try:
                self.s3_client.download_file(self.bucket_name, file_key, file_name)
                print(f"Baixado: {file_key} -> {file_name}")
            except Exception as e:
                print(f"Erro ao baixar {file_key}: {e}")
    
    def concatenate_csv_files(self):
        """Lê e concatena os arquivos CSV baixados em um único DataFrame."""
        csv_files = [os.path.join(self.local_dir, f) for f in os.listdir(self.local_dir) if f.endswith('.csv')]
        df_list = []
        for file in csv_files:
            try:
                df = pd.read_csv(file)
                df_list.append(df)
            except Exception as e:
                print(f"Erro ao ler {file}: {e}")
        
        if df_list:
            return pd.concat(df_list, ignore_index=True)
        else:
            return pd.DataFrame()


    def download_from_s3(self):
        """
        Faz o download de um arquivo do S3.
        
        :param bucket_name: Nome do bucket no S3.
        :param object_key: Caminho/Chave do arquivo no S3.
        :param download_path: Caminho local onde o arquivo será salvo.
        """
      
        try:
            # Fazendo o download do arquivo
            self.s3_client.download_file(self.bucket_name, self.arq, self.new_arq)
            print(f"Download concluído: {self.local_dir}")
        except Exception as e:
            print(f"Erro ao fazer download do arquivo: {e}")

print("Done")

Done


### Download dos arquivos no S3 para a pasta de trabalho - Noticias

In [5]:

bucket_name = "datathon-base"
prefix = "base_original/noticias/"
arq = 'base_original/noticias/dados_2022_08.csv'
local_dir = '/kaggle/working/noticias/'
new_arq = '/kaggle/working/noticias/dados_noticias_2022_08.csv'
downloader = S3FileDownloader(bucket_name, prefix, local_dir, arq, new_arq)
downloader.download_from_s3()


Download concluído: /kaggle/working/noticias/


### Carregando noticias de 08/22

In [6]:
df_news = pd.read_csv('/kaggle/working/noticias/dados_noticias_2022_08.csv')

In [7]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10979 entries, 0 to 10978
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   page      10979 non-null  object
 1   url       10979 non-null  object
 2   issued    10979 non-null  object
 3   modified  10979 non-null  object
 4   title     10979 non-null  object
 5   body      10979 non-null  object
 6   caption   10979 non-null  object
dtypes: object(7)
memory usage: 600.5+ KB


In [8]:
df_news = df_news.dropna()

In [9]:
df_news.head(3)

,page,url,issued,modified,title,body,caption
0,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez..."
1,a7ca6fc2-c287-4b46-b056-30b93f98abab,http://g1.globo.com/rn/rio-grande-do-norte/not...,2022-08-01 15:24:32+00:00,2022-08-01 15:24:33+00:00,"Sine-RN tem 199 vagas de emprego para Natal, M...","Sine-RN tem 199 vagas de emprego para Natal, M...","Para concorrer às vagas, o candidato deve se c..."
2,2a357fd7-fa8d-4417-917c-c7150d49409e,http://g1.globo.com/pa/santarem-regiao/noticia...,2022-08-02 17:02:34+00:00,2022-08-02 17:12:19+00:00,"Em Óbidos, Semed confirma para quarta-feira, 8...","Escola José Veríssimo, em Óbidos, preparada pa...",Mais de 6 mil alunos devem retornar às 14 esco...


### Carregando dados das ultimas interações - 08/22

In [10]:
bucket_name = "datathon-base"
prefix = "base_original/interacoes/"
arq = 'base_original/interacoes/dados_2022_08.csv'
local_dir = '/kaggle/working/interacoes/'
new_arq = '/kaggle/working/interacoes/hitorico_user.csv'
downloader = S3FileDownloader(bucket_name, prefix, local_dir, arq, new_arq)
downloader.download_from_s3()

Download concluído: /kaggle/working/interacoes/


In [11]:
df_interacoes = pd.read_csv('/kaggle/working/interacoes/hitorico_user.csv')

In [12]:
df_interacoes.rename(columns={'history': 'page'}, inplace = True) 

In [13]:
df_interacoes.shape

(2779562, 10)

In [14]:
df_interacoes.head(3)

,userId,userType,historySize,page,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,Logged,12,7456c88d-6473-46ed-83a6-dfc406994162,2022-08-01 16:13:12.309,21,84995,43.48,1,2022-08-01 16:13:12.309
1,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,Logged,12,458bf0ec-efb4-4bfd-9446-c80295e6aa87,2022-08-05 11:03:37.494,79,536583,75.56,1,2022-08-05 11:03:37.494
2,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,Logged,12,89fa73f0-4341-4de4-bb2a-e429ef96bd43,2022-08-05 11:25:02.544,25,110000,45.06,1,2022-08-05 11:25:02.544


### Mesclando noticias e interações

In [15]:
df = df_news.merge(df_interacoes, on='page')
df.head(3)

,page,url,issued,modified,title,body,caption,userId,userType,historySize,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez...",4b0d4ab93884470f7764076a5e453dc8e3406c36609f98...,Logged,3,2022-08-03 00:14:09.411,17,28065,62.76,1,2022-08-03 00:14:09.411
1,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez...",89271af972b940723854f16b1400a0d46d13d50613ffdb...,Non-Logged,10,2022-08-02 00:14:17.184,0,40000,50.56,1,2022-08-02 00:14:17.184
2,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez...",651427356537b3b0f5b30d9bd7d6214d0987e3359f3d5c...,Logged,17,2022-08-04 21:08:34.871,9,93727,23.98,4,2022-08-04 21:08:34.871


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307517 entries, 0 to 307516
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   page                     307517 non-null  object 
 1   url                      307517 non-null  object 
 2   issued                   307517 non-null  object 
 3   modified                 307517 non-null  object 
 4   title                    307517 non-null  object 
 5   body                     307517 non-null  object 
 6   caption                  307517 non-null  object 
 7   userId                   307517 non-null  object 
 8   userType                 307517 non-null  object 
 9   historySize              307517 non-null  int64  
 10  timestampHistory         307517 non-null  object 
 11  numberOfClicksHistory    307517 non-null  int64  
 12  timeOnPageHistory        307517 non-null  int64  
 13  scrollPercentageHistory  307517 non-null  float64
 14  page

### Definindo o Score de cada noticia pelo Tempo na página + Numero de cliques na matéria + Scroll na matéria

In [17]:
df['score'] = df['timeOnPageHistory'] + df['numberOfClicksHistory'] + df['scrollPercentageHistory']   

In [18]:
df.head(3)

,page,url,issued,modified,title,body,caption,userId,userType,historySize,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new,score
0,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez...",4b0d4ab93884470f7764076a5e453dc8e3406c36609f98...,Logged,3,2022-08-03 00:14:09.411,17,28065,62.76,1,2022-08-03 00:14:09.411,28144.76
1,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez...",89271af972b940723854f16b1400a0d46d13d50613ffdb...,Non-Logged,10,2022-08-02 00:14:17.184,0,40000,50.56,1,2022-08-02 00:14:17.184,40050.56
2,56ed4cb1-65c2-42f7-80f4-ea73f0aad59c,http://g1.globo.com/ba/bahia/noticia/2022/08/0...,2022-08-01 17:46:54+00:00,2022-08-01 18:10:14+00:00,Censo 2022: coleta de dados em domicílios come...,Censo do IBGE 2022 começa nesta segunda-feira\...,"Nesta edição, o censo trará, pela primeira vez...",651427356537b3b0f5b30d9bd7d6214d0987e3359f3d5c...,Logged,17,2022-08-04 21:08:34.871,9,93727,23.98,4,2022-08-04 21:08:34.871,93759.98


### Agrupando as noticias e somando o score acumulado por notícia

In [19]:
df_grouped = df.groupby(['page', 'issued', 'title'], as_index=False)['score'].sum()


In [20]:
df_grouped.head(3)

,page,issued,title,score
0,0005256a-90fe-4356-a89f-a91cb0057d0f,2022-08-02 00:59:19+00:00,Santuário com restos mortais de Aleijadinho va...,113327.64
1,000b0af5-0476-4e7e-b050-0d4aa0f2ab17,2022-08-04 23:16:32+00:00,"Vacinação contra a Covid: 169,2 milhões de bra...",485804.90
2,000c036e-0640-4cdb-a649-9f144a3d3709,2022-08-01 20:15:21+00:00,Pará têm três casos suspeitos de varíola dos m...,1120400.88


In [21]:
df_grouped.shape

(9921, 4)

### Ordenando por Score e data de publicação

In [22]:
df_sorted = df_grouped.sort_values(by=['score'], ascending=[False])


In [23]:
df_sorted

,page,issued,title,score
9889,ff3e8c61-d72a-42bf-a84a-ecf5409468dd,2022-07-09 13:32:52+00:00,Luva de Pedreiro diz que foi intimado pela Jus...,5.051177e+08
129,037155f4-4db8-4f07-8d0a-3e559605b8ad,2022-07-04 02:08:02+00:00,Luva de Pedreiro sobre a polêmica com o antigo...,4.268555e+08
6190,a25e68dd-1343-4dc5-be73-2ae40aaca5ee,2022-07-05 16:40:02+00:00,Celulares com 5G: veja a lista de aparelhos ho...,3.644834e+08
2951,4e73dd4e-0e2b-4a11-8f5b-658619b08027,2022-07-14 09:23:20+00:00,Fonoaudióloga suspeita de torturar crianças au...,3.438642e+08
2957,4e9c2825-ff13-41ca-8e91-edd848060d19,2022-08-02 12:45:41+00:00,Menina de 10 anos que estava desaparecida após...,2.499455e+08
...,...,...,...,...
9191,ed917d5b-9abb-4a0b-a22c-3e538c5c3a40,2022-08-04 21:33:06+00:00,Servidora da Vara da Família suspeita de cobra...,5.597740e+03
4594,79236d5d-17a2-49b8-9207-9abebf8466a9,2022-08-02 20:17:50+00:00,Com média de 5 ocorrências de queimada por sem...,5.585560e+03
9585,f7ca6362-cf2e-4586-86b7-021e70f7801f,2022-08-04 14:22:28+00:00,Ônibus perde controle e atinge duas caminhonet...,5.563900e+03
1143,1e646c9f-3239-47bf-8701-97ea7b1765f2,2022-08-08 21:05:51+00:00,Moraes pede explicações do governo sobre supos...,5.460690e+03


In [24]:
top_news = df_sorted.head(10)

In [25]:
top_news

,page,issued,title,score
9889,ff3e8c61-d72a-42bf-a84a-ecf5409468dd,2022-07-09 13:32:52+00:00,Luva de Pedreiro diz que foi intimado pela Jus...,5.051177e+08
129,037155f4-4db8-4f07-8d0a-3e559605b8ad,2022-07-04 02:08:02+00:00,Luva de Pedreiro sobre a polêmica com o antigo...,4.268555e+08
6190,a25e68dd-1343-4dc5-be73-2ae40aaca5ee,2022-07-05 16:40:02+00:00,Celulares com 5G: veja a lista de aparelhos ho...,3.644834e+08
2951,4e73dd4e-0e2b-4a11-8f5b-658619b08027,2022-07-14 09:23:20+00:00,Fonoaudióloga suspeita de torturar crianças au...,3.438642e+08
2957,4e9c2825-ff13-41ca-8e91-edd848060d19,2022-08-02 12:45:41+00:00,Menina de 10 anos que estava desaparecida após...,2.499455e+08
7296,bf257382-74fb-4392-ad6a-143240e39f81,2022-08-05 08:23:07+00:00,"Jô Soares, ícone do humor e da TV, morre em Sã...",2.430707e+08
9291,f0a78e58-ec7e-494c-9462-fbd6446a9a89,2022-08-03 20:08:46+00:00,Caso Bárbara: suspeito de envolvimento no assa...,2.428267e+08
3192,54600b87-2432-4147-92b6-a93e2f6befb4,2022-07-05 09:00:43+00:00,Compras no exterior: saiba quando é preciso pa...,1.966580e+08
1170,1f32787b-de2b-49be-8c20-ddaeae34cc22,2022-08-10 09:55:29+00:00,Filha é presa por golpe estimado em R$ 725 mil...,1.963689e+08
5351,8c246d2b-81bd-4c1f-b563-2c905675f984,2022-08-02 14:44:28+00:00,"Voo de Nancy Pelosi pousa em Taiwan, e China f...",1.884798e+08


### Classe de Recomendação

In [26]:
class Recomendation:
    def recomendation_cold_start(df_noticias: pd.DataFrame, df_interacoes: pd.DataFrame):

        df_interacoes.rename(columns={'history': 'page'}, inplace = True) 
        df = df_noticias.merge(df_interacoes, on='page')
    
        #monta o score das noticias
        df['score'] = df['timeOnPageHistory'] + df['numberOfClicksHistory'] + df['scrollPercentageHistory']   
    
        #agrupando as noticias e somando scores
        df_grouped = df.groupby(['page', 'issued', 'title'], as_index=False)['score'].sum()
    
        #ordenando as notiticas pelo score
        df_sorted = df_grouped.sort_values(by=['score'], ascending=[False])
    
        rec = df_sorted.head(10) 
        rec_new = rec[['page', 'score']]
        
        return rec_new

print('Done')
    

Done


In [27]:
recommendations = Recomendation.recomendation_cold_start(df_news, df_interacoes)
print(recommendations)

                                      page         score
9889  ff3e8c61-d72a-42bf-a84a-ecf5409468dd  5.051177e+08
129   037155f4-4db8-4f07-8d0a-3e559605b8ad  4.268555e+08
6190  a25e68dd-1343-4dc5-be73-2ae40aaca5ee  3.644834e+08
2951  4e73dd4e-0e2b-4a11-8f5b-658619b08027  3.438642e+08
2957  4e9c2825-ff13-41ca-8e91-edd848060d19  2.499455e+08
7296  bf257382-74fb-4392-ad6a-143240e39f81  2.430707e+08
9291  f0a78e58-ec7e-494c-9462-fbd6446a9a89  2.428267e+08
3192  54600b87-2432-4147-92b6-a93e2f6befb4  1.966580e+08
1170  1f32787b-de2b-49be-8c20-ddaeae34cc22  1.963689e+08
5351  8c246d2b-81bd-4c1f-b563-2c905675f984  1.884798e+08


### Transformando a recomendação em CSV e enviando para a AWS

In [28]:
recommendations.to_csv('/kaggle/working/recomendations.csv', index=False)

In [29]:
file_path = "/kaggle/working/recomendations.csv"
bucket_name = "datathon-base"
object_name= "recomendations.csv"
upload_to_s3(file_path, bucket_name)

✔ Upload de /kaggle/working/recomendations.csv para s3://datathon-base/recomendations.csv concluído.


True